In [ ]:
import os
import json
from pathlib import Path

def create_manifest_from_medical_voice_data(base_path, output_file="manifest.jsonl"):
    """
    의료진 및 환자 음성 데이터에서 manifest 파일을 생성합니다.
    
    Args:
        base_path (str): 기본 경로 (/home/nas4/DB/AI_Hub_비대면진료/비대면_진료를_위한_의료진_및_환자_음성/Validation/medv)
        output_file (str): 출력할 manifest 파일명
    """
    
    manifest_data = []
    
    # 기본 경로 설정
    base_dir = Path(base_path)
    
    # 간호사, 의사, 환자 폴더 순회
    for role_dir in base_dir.iterdir():
        if role_dir.is_dir():
            print(f"처리 중: {role_dir.name}")
            
            # 각 개별 화자 폴더 순회
            for speaker_dir in role_dir.iterdir():
                if speaker_dir.is_dir():
                    print(f"  화자: {speaker_dir.name}")
                    
                    # 각 화자 폴더 내의 파일들 처리
                    for file_path in speaker_dir.iterdir():
                        # JSON 파일만 처리 (라벨 정보가 있는 파일)
                        if file_path.suffix == '.json':
                            # JSON 파일 읽기
                            with open(file_path, 'r', encoding='utf-8') as f:
                                label_data = json.load(f)
                            
                            # 음성 파일 경로 생성 (JSON과 같은 이름의 WAV 파일)
                            audio_filename = file_path.stem + '.wav'
                            audio_path = file_path.parent / audio_filename
                            
                            # 음성 파일이 실제로 존재하는지 확인 (선택사항)
                            # if not audio_path.exists():
                            #     continue
                            
                            # 전사 정보 추출
                            transcript = label_data.get('전사정보', {}).get('LabelText', '')
                            
                            # 파일 정보 추출
                            file_info = label_data.get('파일정보', {})
                            duration = float(file_info.get('FileLength', 0))
                            
                            # 화자 정보 추출
                            speaker_info = label_data.get('화자정보', {})
                            gender = speaker_info.get('Gender', '')
                            age = speaker_info.get('Age', '')
                            region = speaker_info.get('Region', '')
                            
                            # Manifest 엔트리 생성
                            manifest_entry = {
                                "audio_filepath": str(audio_path),
                                "text": transcript,
                                "duration": duration,
                                "speaker_id": speaker_dir.name,
                                "role": role_dir.name,
                                "gender": gender,
                                "age": age,
                                "region": region
                            }
                            
                            manifest_data.append(manifest_entry)
    
    # Manifest 파일 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        for entry in manifest_data:
            f.write(json.dumps(entry, ensure_ascii=False) + '\n')
    
    print(f"Manifest 파일이 생성되었습니다: {output_file}")
    print(f"총 {len(manifest_data)}개의 음성 파일이 처리되었습니다.")
    
    return manifest_data

# 실행
base_path = "dataset/medv"  # 상대경로로 변경
manifest_data = create_manifest_from_medical_voice_data(base_path)


In [1]:
# 개선된 버전 - 진행률 표시 및 오류 처리 포함
import os
import json
from pathlib import Path
from tqdm import tqdm

def create_manifest_with_progress(base_path, output_file="manifest.jsonl"):
    """
    진행률 표시와 함께 manifest 파일을 생성합니다.
    """
    
    manifest_data = []
    base_dir = Path(base_path)
    
    # 전체 파일 수 계산
    total_json_files = 0
    all_files = []
    
    print("파일 수 계산 중...")
    for role_dir in base_dir.iterdir():
        if role_dir.is_dir():
            for speaker_dir in role_dir.iterdir():
                if speaker_dir.is_dir():
                    json_files = list(speaker_dir.glob("*.json"))
                    all_files.extend(json_files)
                    total_json_files += len(json_files)
    
    print(f"총 {total_json_files}개의 JSON 파일을 처리합니다.")
    
    # 프로그레스 바를 사용하여 파일 처리
    errors = []
    
    with tqdm(total=total_json_files, desc="Manifest 생성") as pbar:
        for file_path in all_files:
            # JSON 파일 처리
            role_name = file_path.parent.parent.name
            speaker_name = file_path.parent.name
            
            # JSON 파일 읽기 및 처리
            with open(file_path, 'r', encoding='utf-8') as f:
                label_data = json.load(f)
            
            # 음성 파일 경로 생성
            audio_filename = file_path.stem + '.wav'
            audio_path = file_path.parent / audio_filename
            
            # 전사 정보 추출
            transcript = label_data.get('전사정보', {}).get('LabelText', '')
            
            # 파일 정보 추출
            file_info = label_data.get('파일정보', {})
            duration_str = file_info.get('FileLength', '0')
            
            # duration 처리 (문자열일 수 있음)
            duration = float(duration_str) if duration_str else 0.0
            
            # 화자 정보 추출
            speaker_info = label_data.get('화자정보', {})
            gender = speaker_info.get('Gender', '')
            age = speaker_info.get('Age', '')
            region = speaker_info.get('Region', '')
            
            # Manifest 엔트리 생성
            manifest_entry = {
                "audio_filepath": str(audio_path),
                "text": transcript,
                "duration": duration,
                "speaker_id": speaker_name,
                "role": role_name,
                "gender": gender,
                "age": age,
                "region": region
            }
            
            manifest_data.append(manifest_entry)
            pbar.update(1)
    
    # Manifest 파일 저장
    print(f"\n{output_file}에 저장 중...")
    with open(output_file, 'w', encoding='utf-8') as f:
        for entry in manifest_data:
            f.write(json.dumps(entry, ensure_ascii=False) + '\n')
    
    print(f"✅ Manifest 파일이 생성되었습니다: {output_file}")
    print(f"📊 총 {len(manifest_data)}개의 음성 파일이 처리되었습니다.")
    
    # 통계 정보 출력
    roles = {}
    for entry in manifest_data:
        role = entry['role']
        if role not in roles:
            roles[role] = 0
        roles[role] += 1
    
    print("\n📈 역할별 통계:")
    for role, count in roles.items():
        print(f"  {role}: {count:,}개")
    
    return manifest_data


In [ ]:
# 실행 및 결과 확인
base_path = "dataset/medv"  # 상대경로로 변경

# 개선된 버전으로 실행
print("🚀 개선된 버전으로 Manifest 생성 시작...")
manifest_data = create_manifest_with_progress(base_path, "dataset/medical_manifest.jsonl")

# 샘플 데이터 확인
print("\n📝 생성된 데이터 샘플 (처음 3개):")
for i, entry in enumerate(manifest_data[:3]):
    print(f"\n[{i+1}] {entry['role']} - {entry['speaker_id']}")
    print(f"   텍스트: {entry['text']}")
    print(f"   음성파일: {entry['audio_filepath']}")
    print(f"   길이: {entry['duration']}초")
    print(f"   화자정보: {entry['gender']}, {entry['age']}, {entry['region']}")


🚀 개선된 버전으로 Manifest 생성 시작...
파일 수 계산 중...
총 141819개의 JSON 파일을 처리합니다.


Manifest 생성: 100%|██████████| 141819/141819 [08:34<00:00, 275.44it/s]



medical_voice_manifest.jsonl에 저장 중...
✅ Manifest 파일이 생성되었습니다: medical_voice_manifest.jsonl
📊 총 141819개의 음성 파일이 처리되었습니다.

📈 역할별 통계:
  간호사: 42,177개
  의사: 51,898개
  환자: 47,744개

📝 생성된 데이터 샘플 (처음 3개):

[1] 간호사 - HA_0355
   텍스트: 영에서 십 중에서 통증 정도를 골라 보세요.
   음성파일: /home/nas4/DB/AI_Hub_비대면진료/비대면_진료를_위한_의료진_및_환자_음성/Validation/medv/간호사/HA_0355/HA_0355-1833-01-02-F-05-A.wav
   길이: 3.18초
   화자정보: Female, 30~39, 서울/인천/경기

[2] 간호사 - HA_0355
   텍스트: 그 증상으로 얼마 정도 고생하셨어요?
   음성파일: /home/nas4/DB/AI_Hub_비대면진료/비대면_진료를_위한_의료진_및_환자_음성/Validation/medv/간호사/HA_0355/HA_0355-1884-01-02-F-05-A.wav
   길이: 2.76초
   화자정보: Female, 30~39, 서울/인천/경기

[3] 간호사 - HA_0355
   텍스트: 고혈압도 치료 중이신가요?
   음성파일: /home/nas4/DB/AI_Hub_비대면진료/비대면_진료를_위한_의료진_및_환자_음성/Validation/medv/간호사/HA_0355/HA_0355-1711-01-02-F-05-A.wav
   길이: 2.04초
   화자정보: Female, 30~39, 서울/인천/경기


In [3]:
# 추가 분석 및 필터링 기능
import pandas as pd

def analyze_manifest_data(manifest_data):
    """생성된 manifest 데이터를 분석합니다."""
    
    df = pd.DataFrame(manifest_data)
    
    print("📊 상세 통계 분석")
    print("=" * 50)
    
    # 역할별 통계
    print("\n🎭 역할별 분포:")
    role_counts = df['role'].value_counts()
    for role, count in role_counts.items():
        print(f"  {role}: {count:,}개 ({count/len(df)*100:.1f}%)")
    
    # 성별 분포
    print("\n👥 성별 분포:")
    gender_counts = df['gender'].value_counts()
    for gender, count in gender_counts.items():
        print(f"  {gender}: {count:,}개 ({count/len(df)*100:.1f}%)")
    
    # 연령대 분포
    print("\n🎂 연령대 분포:")
    age_counts = df['age'].value_counts()
    for age, count in age_counts.items():
        print(f"  {age}: {count:,}개 ({count/len(df)*100:.1f}%)")
    
    # 지역별 분포
    print("\n🗺️ 지역별 분포:")
    region_counts = df['region'].value_counts()
    for region, count in region_counts.items():
        print(f"  {region}: {count:,}개 ({count/len(df)*100:.1f}%)")
    
    # 음성 길이 통계
    print("\n⏱️ 음성 길이 통계:")
    print(f"  평균: {df['duration'].mean():.2f}초")
    print(f"  최소: {df['duration'].min():.2f}초")
    print(f"  최대: {df['duration'].max():.2f}초")
    print(f"  전체: {df['duration'].sum():.2f}초 ({df['duration'].sum()/3600:.2f}시간)")
    
    # 텍스트 길이 통계
    df['text_length'] = df['text'].str.len()
    print("\n📝 텍스트 길이 통계:")
    print(f"  평균: {df['text_length'].mean():.1f}자")
    print(f"  최소: {df['text_length'].min()}자")
    print(f"  최대: {df['text_length'].max()}자")
    
    return df

def create_filtered_manifest(df, filters, output_file):
    """조건에 맞는 데이터만 필터링하여 새로운 manifest 파일 생성"""
    
    filtered_df = df.copy()
    
    # 필터 적용
    for key, values in filters.items():
        if key in df.columns:
            if isinstance(values, list):
                filtered_df = filtered_df[filtered_df[key].isin(values)]
            else:
                filtered_df = filtered_df[filtered_df[key] == values]
    
    print(f"필터링 결과: {len(filtered_df):,}개 데이터")
    
    # 필터링된 데이터를 manifest 파일로 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        for _, row in filtered_df.iterrows():
            entry = row.to_dict()
            # text_length 컬럼 제거 (manifest에 불필요)
            if 'text_length' in entry:
                del entry['text_length']
            f.write(json.dumps(entry, ensure_ascii=False) + '\n')
    
    print(f"필터링된 manifest 파일 저장: {output_file}")
    
    return filtered_df


In [ ]:
# 데이터 분석 및 필터링 예시
if 'manifest_data' in locals():
    # 상세 분석 실행
    df = analyze_manifest_data(manifest_data)
    
    print("\n" + "="*50)
    print("🔍 필터링 예시")
    print("="*50)
    
    # 예시 1: 여성 의사 데이터만 추출
    filters_female_doctor = {
        'role': '의사',
        'gender': 'Female'
    }
    filtered_df1 = create_filtered_manifest(df, filters_female_doctor, "female_doctor_manifest.jsonl")
    
    # 예시 2: 20-30대 환자 데이터만 추출
    filters_young_patient = {
        'role': '환자',
        'age': ['20~29', '30~39']
    }
    filtered_df2 = create_filtered_manifest(df, filters_young_patient, "young_patient_manifest.jsonl")
    
    # 예시 3: 특정 지역 데이터만 추출
    filters_region = {
        'region': '서울/경기/인천'
    }
    if len(df[df['region'] == '서울/경기/인천']) > 0:
        filtered_df3 = create_filtered_manifest(df, filters_region, "seoul_region_manifest.jsonl")
    else:
        print("서울/경기/인천 지역 데이터가 없습니다.")
else:
    print("먼저 manifest_data를 생성해주세요.")
